# Cuaderno de ingesta de datos
En este bloque traeremosla información desde datos abiertos.

In [0]:
# Celda 1: Leer datos desde las APIs
url_secop = "/Volumes/main/diplmado_datos/manual/df_secop.csv"
url_men = "/Volumes/main/diplmado_datos/manual/df_men.csv"

# Usamos spark.read para ingerir los datos.
# option("header", "true") indica que la primera fila contiene los nombres de las columnas.
# option("inferSchema", "true") le pide a Spark que adivine el tipo de cada columna.
df_secop = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(url_secop)
df_men = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(url_men)

# display() es un comando especial de Databricks para una visualización interactiva.
print("Datos del SECOP cargados:")
display(df_secop)

print("Datos del MEN cargados:")
display(df_men)

In [0]:
# Celda 2: Guardar los DataFrames como tablas Delta
# La función .saveAsTable() guarda los datos y registra la tabla en el Unity Catalog.
# El modo "overwrite" reemplaza la tabla si ya existe, ideal para actualizaciones.
df_secop.write.format("delta").mode("overwrite").saveAsTable("main.diplmado_datos.secop")
df_men.write.format("delta").mode("overwrite").saveAsTable("main.diplmado_datos.men_estadisticas")

print("¡Tablas guardadas exitosamente en el catálogo 'main', esquema 'diplmado_datos'!")

In [0]:
# Paso 1: Descargar los datos con requests y leerlos en pandas
import requests
import pandas as pd
from io import StringIO
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

url_secop = "https://www.datos.gov.co/resource/rpmr-utcd.csv?$limit=100000"
url_men = "https://www.datos.gov.co/resource/nudc-7mev.csv?$limit=100000"

# Descargar contenido
response_secop = requests.get(url_secop)
response_men = requests.get(url_men)

# Convertir contenido a pandas usando StringIO
df_secop_pd = pd.read_csv(StringIO(response_secop.text))
df_men_pd = pd.read_csv(StringIO(response_men.text))

# Convertir pandas a Spark
df_secop = spark.createDataFrame(df_secop_pd)
df_men = spark.createDataFrame(df_men_pd)

# Mostrar en Databricks
display(df_secop)
display(df_men)

In [0]:
# Paso 1: Descargar los datos con requests y leerlos en pandas
import requests
import pandas as pd
from io import StringIO
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

url_secop = "https://www.datos.gov.co/resource/rpmr-utcd.csv?$limit=100000&$offset=100000"

# Descargar contenido
response_secop = requests.get(url_secop)

# Convertir contenido a pandas usando StringIO
df_secop_pd = pd.read_csv(StringIO(response_secop.text), delimiter=',', low_memory=False)

# limpiar nombres de columnas (opcional pero recomendado)
df_secop_pd.columns = [col.strip().lower().replace(' ', '_') for col in df_secop_pd.columns]

# Verifica que los datos se cargaron correctamente en pandas 
print(df_secop_pd.head())

# Convertir pandas a Spark
df_secop = spark.createDataFrame(df_secop_pd)

# Mostrar en Databricks
display(df_secop)

### Datasets

In [0]:
from pyspark.sql.functions import col

target_schema = spark.table("main.diplmado_datos.secop").schema

df_secop_aligned = df_secop.select(
    [col(field.name).cast(field.dataType) for field in target_schema.fields if field.name in df_secop.columns]
)

df_secop_aligned.write.format("delta") \
    .mode("append") \
    .option("mergeSchema", "true") \
    .saveAsTable("main.diplmado_datos.secop")
    


In [0]:
# registros en SECOP
total_registros = 19446266
# registros actuales 200.000
offset_inicial = 200000
# tamaño
limite = 100000

# bloques faltantes 
paginas_faltantes = ((total_registros - offset_inicial) // limite) + 1

print(f"Quedan {paginas_faltantes} bloques por descargar...")

In [0]:
import requests
import pandas as pd
from io import StringIO
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Crear sesión Spark (si no existe)
spark = SparkSession.builder.getOrCreate()

# Parámetros
total_registros = 19_449_349   # Total estimado de registros en SECOP
limite = 50_000                # Límite máximo por descarga (seguro con datos.gov.co)
paginas = (total_registros // limite) + 1

print(f"Descargando {total_registros:,} registros en {paginas} páginas...")

for i in range(paginas):
    offset = i * limite
    url_secop = f"https://www.datos.gov.co/resource/rpmr-utcd.csv?$limit={limite}&$offset={offset}"

    print(f"→ Página {i+1}/{paginas} (offset={offset})")

    response = requests.get(url_secop)

    if response.status_code == 200:
        df_pd = pd.read_csv(StringIO(response.text), dtype=str, low_memory=False)

        if df_pd.empty:
            print(f"Página {i+1} vacía. Deteniendo descarga.")
            break

        # Normalizar nombres de columnas (opcional)
        df_pd.columns = [col.strip().lower().replace(' ', '_') for col in df_pd.columns]

        # Convertir a Spark
        df_spark = spark.createDataFrame(df_pd)

        # Alinear con el esquema si ya existe la tabla
        if i == 0:
            # Primera página: sobrescribimos y registramos tabla
            df_spark.write.format("delta") \
                .mode("overwrite") \
                .option("overwriteSchema", "true") \
                .saveAsTable("main.diplmado_datos.secop")
        else:
            # Páginas siguientes: alineamos esquema y hacemos append
            target_schema = spark.table("main.diplmado_datos.secop").schema
            df_aligned = df_spark.select(
                [col(field.name).cast(field.dataType) for field in target_schema.fields if field.name in df_spark.columns]
            )

            df_aligned.write.format("delta") \
                .mode("append") \
                .option("mergeSchema", "true") \
                .saveAsTable("main.diplmado_datos.secop")

        print(f"✓ Página {i+1} guardada con {len(df_pd):,} filas.")
    else:
        print(f"Error HTTP {response.status_code} en página {i+1}. Deteniendo proceso.")
        break

print("¡Carga completa de SECOP en Delta!")

In [0]:
token = dbutils.widgets.get("token_app")
print(token)